In [1]:
import pandas as pd
import numpy as np
import csv
import json
import string
import unicodedata
import re
#import folium
import geopy
import geopy.distance
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from scipy.spatial.distance import cosine
from geopy.geocoders import Nominatim
from geopy.distance import great_circle

In [15]:
with open('inv_indx.json', 'r') as fp:
    inv_indx = json.load(fp)
inv_indx={int(key):inv_indx[key] for key in inv_indx}

In [16]:
with open('termiddic.json', 'r') as fp:
    termiddic = json.load(fp)

In [18]:
q1=input()

kitchen house light private


In [29]:
#from finalADM33.ipynb import filtering

In [19]:
from Query import stem_sentences,remove_punctuations,remove_accents

In [20]:
q=stem_sentences(q1)
q=remove_punctuations(q)
q=remove_accents(q)

In [21]:
stop = stopwords.words('english')
stopword=lambda l1: ' '.join([word for word in l1.split() if word not in (stop)])
punctuation= lambda x:''.join([i for i in x if i not in string.punctuation])

In [22]:
q=stopword(q)
q1=punctuation(q)

In [10]:
q1

'kitch hous privat'

In [23]:
tid1=[]
for l in (q1.split()):
    if l in termiddic:
        value=termiddic[l]
        tid1.append(value)

In [24]:
docid1=[]
for t in tid1:
    value=inv_indx[t]
    docid1.append(set(value))
u = set.intersection(*docid1)

In [25]:
u

{597, 3415, 5453, 8159, 11163, 11359, 11447, 15958, 16060, 16617, 17683}

In [26]:
p=pd.DataFrame()
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for b in u:
    d=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas4/doc_"+str(b)+".tsv",names=l,delimiter="\t")
    p=p.append(d[['description','title','city','url']])

In [27]:
p

,description,title,city,url
16621,Hey my place is 10 miles to Circuit of the Am...,Cozy Room with light Breakfast,Austin,https://www.airbnb.com/rooms/15274066?location...
5455,HOLIDAYS LOCAL EVENTS HIGH DEMAND WEEKENDS ...,2 blocks to Rainey! Walk downtown!,Austin,https://www.airbnb.com/rooms/1250575?location=...
17688,Unattached studio on quiet street; walking dis...,Funky Barton Hills Cabin,Austin,https://www.airbnb.com/rooms/14274652?location...
598,Hey my place is 10 miles to Circuit of the Am...,Cozy Room with light breakfast,Austin,https://www.airbnb.com/rooms/15274260?location...
15962,Hey my place is 10 miles to Circuit of the Am...,Cozy Room with light breakfast,Austin,https://www.airbnb.com/rooms/15274260?location...
11451,Guest house in the backyard of residence. Priv...,Cozy Cottage,Hurst,https://www.airbnb.com/rooms/19249179?location...
8162,Hey my place is 10 miles to Circuit of the Am...,Cozy Room with light Breakfast,Austin,https://www.airbnb.com/rooms/15274066?location...
11167,Beautiful bright and charming 600 Square foot...,Beautiful bright and cozy guesthouse availabl...,Georgetown,https://www.airbnb.com/rooms/18749202?location...
16064,Historic Houston Heights Carriage House. Priva...,Historic Houston Heights Norhill,Houston,https://www.airbnb.com/rooms/9631890?location=...
3416,Enjoy this beautiful 2 bedroom house situated ...,Wine Trail Panoramic View Getaway,Fredericksburg,https://www.airbnb.com/rooms/5405790?location=...


In [28]:
with open('new_invindx.json', 'r') as fp:
    new_invindx1= json.load(fp)
new_invindx1={int(key):new_invindx1[key] for key in new_invindx1}

new_invindx=new_invindx1

In [29]:
termdf=pd.read_csv("termdf.csv")
termdf.set_index('word',inplace=True)

In [30]:
d1={}
for h in q1.split():
    c1=q1.count(h)
    tf=c1/len(q1)
    if h in termiddic:
        d1[termiddic[h]]=(termdf.loc[h]['idf'])*tf
    else:
        continue

In [31]:
d1

{25778: 0.04390287848961353,
 26675: 0.060613745892270796,
 27110: 0.11392015602634313,
 30925: 0.051225960678390428}

In [32]:
df1=pd.read_csv("new_backup_title_desc.csv")
df1=df1.fillna("TEXAS")
dn = df1['title']+' '+df1['description']
d2 = pd.DataFrame(dn)
d2.columns = ["words"]
w=d2['words'].str.split(' ', expand=True).stack().value_counts()
wordlist=list(w.index)

In [33]:
# Calculating the Cosine Similarity Ranked by their Score
"""in the following script we calculate the cosine similarity between query and documents that contain all the 
                            words of the query"""
""" based on the formula of cosine similarity in the following script we  should have two vectors of the same size
    therefore we assign a key(here is termid) in the dictionary with the value 0 if the term is not in the query
    but it is in the document"""
precosl={}
finalcosl={}
for d in list(u):
    temp=d1
    g=d2.loc[(d-1),'words']
    g=g.split(" ")
    for i in g:
        if i in termiddic:
            k=termiddic[i]
            tl=new_invindx[k]
            for r in tl:
                if r[0]==d:
                    precosl[termiddic[i]]=r[1]
    for m in precosl:
        if m in temp:
            continue
        else:
            temp[m]=0
    for m in temp:
        if m in precosl:
            continue
        else:
            precosl[m]=0
    doc_d= pd.Series(precosl)
    query_d = pd.Series(temp)
    finalcosl[d]=(1 - cosine(doc_d, query_d))

{}

In [34]:
p1=pd.DataFrame()
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for b in u:
    d=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas5/doc_"+str(b)+".tsv",names=l,delimiter="\t")
    d['Cosine-Similiarty']=finalcosl[b]
    p1=p1.append(d[['description','title','city','url','Cosine-Similiarty']])

In [35]:
p1

,description,title,city,url,Cosine-Similiarty
0,hey place mile circuit america mile airport ...,cozi room light breakfast,Austin,https://www.airbnb.com/rooms/15274066?location...,0.287906
0,holiday local event high demand weekend date w...,block rainey walk downtown,Austin,https://www.airbnb.com/rooms/1250575?location=...,0.035445
0,unattach studio quiet street walk distanc bart...,funki barton hill cabin,Austin,https://www.airbnb.com/rooms/14274652?location...,0.130300
0,hey place mile circuit america mile airport ...,cozi room light breakfast,Austin,https://www.airbnb.com/rooms/15274260?location...,0.167106
0,hey place mile circuit america mile airport ...,cozi room light breakfast,Austin,https://www.airbnb.com/rooms/15274260?location...,0.167106
0,guest hous backyard resid privat entranc park ...,cozi cottag,Hurst,https://www.airbnb.com/rooms/19249179?location...,0.125893
0,hey place mile circuit america mile airport ...,cozi room light breakfast,Austin,https://www.airbnb.com/rooms/15274066?location...,0.151046
0,beauti bright charm squar foot guest hous ava...,beauti bright cozi guesthous avail rent,Georgetown,https://www.airbnb.com/rooms/18749202?location...,0.100646
0,histor houston height carriag hous privat elec...,histor houston height norhil,Houston,https://www.airbnb.com/rooms/9631890?location=...,0.154684
0,enjoy beauti bedroom hous situat top ridg de...,wine trail panoram view getaway,Fredericksburg,https://www.airbnb.com/rooms/5405790?location=...,0.056639


In [36]:
Df=pd.read_csv("Defining_newscore.csv")

In [37]:
#Cleaning the data and extracting meaningful information to define New Score based on City,No of Bedrooms & Average rate/Night
#we remove the $ sign in average_rate_per_night and convert it to float
Df['average_rate_per_night']=Df['average_rate_per_night'].apply(lambda x : int(x.replace('$','')))
Df['bedrooms_count']=Df['bedrooms_count'].apply(lambda x : str(x))
#we convert the bedroom to float and then we replace the Studio with 1
Df['bedrooms_count']=Df['bedrooms_count'].apply(lambda x : float(x.replace('Studio','1')))

In [47]:
c=input('please Enter city: ')

please Enter city: Austin


In [39]:
b=input('please Enter the number of bedrooms? ')

please Enter the number of bedrooms? 4


In [40]:
if b !="Studio":
    b=int(b)
if b=="Studio":
    b=1

In [41]:
r=input('please Enter the number of average_rate_per_night (in dollar)? ')
r=int(r.replace('$',''))

please Enter the number of average_rate_per_night (in dollar)? 50


In [42]:
"""u is the intersection of documents that contains the word from previous part in other words it contains all 
the documents that contains all the words in the query"""
u = [x-1 for x in u]
S=Df.ix[u]

In [43]:
S=S.reset_index(drop=True)

In [48]:
"""Using Builtin library Geopy & Nominatim in order to find the distance between the city that is in the query
and the city in the documents"""

geolocator = Nominatim(user_agent='myapplication')
try:
    location = geolocator.geocode(c,timeout=None)
    lan=location.latitude
    lon=location.longitude
except GeocoderTimedOut as e:
    lan=100
Y=(lan,lon)

In [49]:
w=[0.1,0.3,0.6,0.4]
for i in range(len(S)):
    m=[]
    if b-1<=S.loc[i,'bedrooms_count']<=b+1:
        m.append(1)
    else:
        m.append(0)
    if r-20<=S.loc[i,'average_rate_per_night']<=r+10:
         m.append(1)
    else:
        m.append(0)
    if c==S.loc[i,'city']:
        m.append(1)
        m.append(0)
    else:
        m.append(0)
        X=(S.loc[i,'latitude'],S.loc[i,'longitude'])
        Dist=great_circle(X,Y).km
        if Dist<=10:
            m.append(1)
        else:
            m.append(0)
    DotProduct = np.dot(m,w)
    S.loc[i,'Score']=DotProduct

In [50]:
S.sort_values(by=['Score'],ascending=False, kind='heapsort')

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,Score
3,596,30,1.0,Austin,September 2016,hey place mile circuit america mile airport ...,30.238889,-97.578815,cozi room light breakfast,https://www.airbnb.com/rooms/15274260?location...,0.9
6,8158,30,1.0,Austin,September 2016,hey place mile circuit america mile airport ...,30.240384,-97.580312,cozi room light breakfast,https://www.airbnb.com/rooms/15274066?location...,0.9
0,16616,30,1.0,Austin,September 2016,hey place mile circuit america mile airport ...,30.240384,-97.580312,cozi room light breakfast,https://www.airbnb.com/rooms/15274066?location...,0.9
4,15957,30,1.0,Austin,September 2016,hey place mile circuit america mile airport ...,30.238889,-97.578815,cozi room light breakfast,https://www.airbnb.com/rooms/15274260?location...,0.9
2,17682,125,1.0,Austin,July 2016,unattach studio quiet street walk distanc bart...,30.256405,-97.778506,funki barton hill cabin,https://www.airbnb.com/rooms/14274652?location...,0.6
1,5452,109,2.0,Austin,February 2011,holiday local event high demand weekend date w...,30.257062,-97.733581,block rainey walk downtown,https://www.airbnb.com/rooms/1250575?location=...,0.6
10,11358,1100,4.0,Buda,October 2012,eleg bedroom bath sq ft hous quiet shadi cu...,30.114181,-97.857215,min austin formula sxsw,https://www.airbnb.com/rooms/762234?location=B...,0.1
7,11162,105,1.0,Georgetown,May 2017,beauti bright charm squar foot guest hous ava...,30.639548,-97.718014,beauti bright cozi guesthous avail rent,https://www.airbnb.com/rooms/18749202?location...,0.0
8,16059,70,1.0,Houston,March 2015,histor houston height carriag hous privat elec...,29.793702,-95.385409,histor houston height norhil,https://www.airbnb.com/rooms/9631890?location=...,0.0
9,3414,160,2.0,Fredericksburg,December 2014,enjoy beauti bedroom hous situat top ridg de...,30.168097,-98.819464,wine trail panoram view getaway,https://www.airbnb.com/rooms/5405790?location=...,0.0
